## Risk as Function of Sensitivity and Specificity

I noticed that in the plot for the risk in LNL IV as a function of sensitivity and specificity, the risk goes down and up again with a rising specificity. Here, I wanted to investigate how that can be.

<img src="../tex/figures/wg_sens_spec_risks.png" width="700">

For illustrative purposes, we can look at a simpler model with only the LNLs III and IV:

```mermaid
flowchart LR
    tumor((T)) --> lnl3((III))
    tumor((T)) --> lnl4((IV))
    lnl3((III)) --> lnl4((IV))
```

In this case, the risk $P( X_4 = 🟢 \mid D_3 = 🔴, D_4 = 🟢 )$ can be written as

$$
\begin{aligned}
P( X_4 = 🟢 \mid D_3 = 🔴, D_4 = 🟢 ) &= \frac{
    \sum_{x_3 \in \{ 🟢,🔴 \}}{ P( D_3 = 🔴 \mid X_3 = x_3 ) \cdot P( D_4 = 🟢 \mid X_4 = 🔴 ) \cdot P(X_3 = x_3, X_4 = 🔴) }
}{
    \sum_{\substack{x_3 \in \{ 🟢,🔴 \} \\ x_4 \in \{ 🟢,🔴 \}}}{ P( D_3 = 🔴 \mid X_3 = x_3 ) \cdot P( D_4 = 🟢 \mid X_4 = x_4 ) \cdot P(X_4 = x_4) }
} \\
&= \frac{
    (1 - s_P) \cdot \left[ (1 - s_N) \cdot P(X_3 = 🟢, X_4 = 🔴) + s_N \cdot P(X_3 = 🔴, X_4 = 🔴) \right]
}{
    (1 - s_P) \cdot \left[ (1 - s_N) \cdot P(X_3 = 🟢, X_4 = 🔴) + s_N \cdot P(X_3 = 🔴, X_4 = 🔴) \right]
    + s_P \cdot \left[ (1 - s_P) \cdot P(X_3 = 🟢, X_4 = 🟢) + s_N \cdot P(X_3 = 🔴, X_4 = 🟢) \right]
} \\
&= \frac{
    \alpha + \beta - \alpha \cdot s_P
}{
    \alpha + \beta + \left( \delta + \gamma - \alpha \right) \cdot s_P - \gamma \cdot s_P^2
}
\end{aligned}
$$

Which we can now plot for varying values of $\alpha, \beta, \gamma \in [0,1]$:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def risk(d3, d4, x3, x4, marg3=0.1, marg4=0.5):
    d3 = int(d3)
    d4 = int(d4)
    x3 = int(x3)
    x4 = int(x4)

    prior = np.outer(
        [1. - marg3, marg3],
        [1. - marg4, marg4],
    )
    def inner(spec, sens):
        spsn = np.diag([spec, sens]) + np.diag([1. - spec, 1. - sens])[::-1]
        enum = prior[x4,x3] * spsn[d3,x3] * spsn[d4,x4]
        denom = np.sum(prior[:,:] * spsn[d3,:] * spsn[d4,:])
        return enum / denom
    return inner

In [ ]:
sens = 0.5

risk3and4_given3not4 = risk(d3=True, d4=False, x3=True, x4=True)
risknot3and4_given3not4 = risk(d3=True, d4=False, x3=False, x4=True)

specs = np.linspace(0.5, 1., 100)
y = np.empty_like(specs)
for i,spec in enumerate(specs):
    y[i] = risk3and4_given3not4(spec, sens) + risknot3and4_given3not4(spec, sens)

plt.plot(specs, y)

In [ ]:
def ana_risk(spec, alpha=None, beta=None, gamma=None, delta=None):
    alpha = np.random.uniform() if alpha is None else alpha
    beta = np.random.uniform() if beta is None else beta
    gamma = np.random.uniform() if gamma is None else gamma
    delta = np.random.uniform() if delta is None else delta

    return (alpha + beta - alpha * spec) / (alpha + beta + (delta + gamma - alpha) * spec - gamma * spec**2)

In [ ]:
specs = np.linspace(0.5, 1., 100)

plt.plot(specs, ana_risk(specs))